In [2]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    #temperature=0,
    #max_tokens=None,
    #timeout=None,
    max_retries=2,
    # api_key="...",
    # other params...
)
model

ChatDeepSeek(client=<openai.resources.chat.completions.completions.Completions object at 0x00000298DC568AD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000298DC642060>, root_client=<openai.OpenAI object at 0x00000298D8699B80>, root_async_client=<openai.AsyncOpenAI object at 0x00000298DC641FA0>, model_name='deepseek-chat', model_kwargs={}, max_retries=2, api_key=SecretStr('**********'), api_base='https://api.deepseek.com/v1')

In [9]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

agent = create_agent(
    model=model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)
result["messages"]


[HumanMessage(content="What's my current balance?", additional_kwargs={}, response_metadata={}, id='853e2ce7-1731-445e-a2a5-d7e8121f75b8'),
 AIMessage(content="I'll check your account information to get your current balance.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 144, 'total_tokens': 164, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 16}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'd6165706-b065-4b68-b8a9-8737f4a5c892', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--24f87c3f-1296-407c-a6a2-885e78b23cca-0', invalid_tool_calls=[{'type': 'invalid_tool_call', 'id': 'call_00_w1UYIR9jXM1voAKAFcwCNNjT', 'name': 'get_account_info', 'args': '', 'error': 'Function get_account_info arguments:\n\n\n\nare no

In [14]:
from pathlib import Path
@tool
def list_files_tool(directory_path: str = ".") -> str:
    """列出指定目录中的所有文件和子目录。
    
    Args:
        directory_path: 要列出的目录路径，默认为当前目录
    
    Returns:
        目录中的文件和子目录列表
    """
    try:
        path = Path(directory_path)
        if not path.exists():
            return f"错误：目录 '{directory_path}' 不存在"
        
        if not path.is_dir():
            return f"错误：'{directory_path}' 不是一个目录"
        
        items = []
        for item in sorted(path.iterdir()):
            item_type = "📁" if item.is_dir() else "📄"
            items.append(f"{item_type} {item.name}")
        
        if not items:
            return f"目录 '{directory_path}' 是空的"
        
        return f"目录 '{directory_path}' 的内容：\n" + "\n".join(items)
    except Exception as e:
        return f"列出目录 '{directory_path}' 时出错: {str(e)}"

tools = [list_files_tool, get_account_info]
agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "列出当前目录，并且查询我的账单"
        }
    ]
}
result = agent.invoke(Message,
                      context = UserContext(user_id="user123"))
result["messages"]

[HumanMessage(content='列出当前目录，并且查询我的账单', additional_kwargs={}, response_metadata={}, id='4d500b80-07bc-4b2f-a97f-68f076976f92'),
 AIMessage(content='我来帮您列出当前目录并查询账单信息。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 221, 'total_tokens': 249, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 29}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'a98702fa-ab34-4fe6-aea1-00f60e2de751', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--e51296bc-82e4-43ab-bec9-77be24bd605c-0', tool_calls=[{'name': 'list_files_tool', 'args': {}, 'id': 'call_00_6NEILvAT4iLv77QEaa5INMXU', 'type': 'tool_call'}, {'name': 'get_account_info', 'args': {}, 'id': 'call_01_jouhcxRRw3Cqu5j1DyqMCqNz', 'type': 'tool_call'}], usage_metadata={'input_toke

In [17]:
@tool
def read_file_tool(file_path: str) -> str:
    """读取文件内容。输入文件路径，返回文件的文本内容。
    
    Args:
        file_path: 要读取的文件路径（相对路径或绝对路径）
    
    Returns:
        文件内容的字符串，如果出错则返回错误信息
    """
    try:
        path = Path(file_path)
        if not path.exists():
            return f"错误：文件 '{file_path}' 不存在"
        
        if not path.is_file():
            return f"错误：'{file_path}' 不是一个文件"
        
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        return f"文件 '{file_path}' 的内容：\n{content}"
    except UnicodeDecodeError:
        return f"错误：无法读取文件 '{file_path}'，可能是二进制文件"
    except Exception as e:
        return f"读取文件 '{file_path}' 时出错: {str(e)}"
    
tools = [list_files_tool, read_file_tool]

agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "读取文件source\test_file_tools.py的内容 "
        }
    ]
}
result = agent.invoke(Message, context = UserContext(user_id="user123"))
result['messages']


[HumanMessage(content='读取文件source\test_file_tools.py的内容 ', additional_kwargs={}, response_metadata={}, id='2f77877b-bbd4-42aa-a4c3-7d9ccbbdd13c'),
 AIMessage(content='我来读取文件 `source est_file_tools.py` 的内容。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 282, 'total_tokens': 317, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 26}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '29b14597-c2f8-4696-83d2-3b22544b9e11', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--212c4d1b-b6a3-4370-be1e-e02a9e82bc0b-0', tool_calls=[{'name': 'read_file_tool', 'args': {'file_path': 'source est_file_tools.py'}, 'id': 'call_00_YT60bQZqn0rtepTlck2nL3xb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 282, 'output_tokens': 35

In [19]:
import pandas as pd
import numpy as np
from typing import List, Optional, Dict, Any
from sklearn.tree import DecisionTreeClassifier
from langchain.tools import tool

# ============== 1. Binning helper =====================
@tool
def bin_single_feature_tool(
    series: pd.Series,
    y: Optional[pd.Series] = None,
    method: str = "quantile",   # 'quantile' | 'width' | 'tree'
    n_bins: int = 10,
    min_leaf_frac: float = 0.05,
) -> pd.Series:
    """
    Bin a single numeric feature according to the specified method.

    Parameters
    ----------
    series : pd.Series
        Raw feature values.
    y : pd.Series, optional
        Target labels (only needed when method='tree').
    method : str
        'quantile' (equal frequency), 'width' (equal width),
        or 'tree' (decision-tree-based optimal binning).
    n_bins : int
        Number of bins or max leaf nodes (for tree).
    min_leaf_frac : float
        Minimum fraction of samples per leaf for tree binning.

    Returns
    -------
    pd.Series
        Series of bin labels (string), same index as input series.
        Missing values are grouped into 'MISSING'.
    """
    # convert to numeric, coerce invalid to NaN
    s = pd.to_numeric(series, errors="coerce")
    bins = pd.Series(index=s.index, dtype="object")

    # handle missing values as a separate bin
    missing_mask = s.isna()

    # ================== quantile / width ==================
    if method in ("quantile", "width"):
        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            # all values are missing
            bins[missing_mask] = "MISSING"
            return bins

        s_valid = s[valid_mask]

        try:
            if method == "quantile":
                # equal frequency binning
                binned = pd.qcut(s_valid, q=n_bins, duplicates="drop")
            else:
                # equal width binning
                binned = pd.cut(s_valid, bins=n_bins, duplicates="drop")
        except ValueError:
            # e.g. all values are identical
            binned = pd.Series(["ALL"] * valid_mask.sum(), index=s_valid.index)

        bins[valid_mask] = binned.astype(str)
        bins[missing_mask] = "MISSING"
        return bins

    # ================== tree-based binning ==================
    elif method == "tree":
        if y is None:
            raise ValueError("y must be provided when method='tree'.")

        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            bins[missing_mask] = "MISSING"
            return bins

        X = s[valid_mask].values.reshape(-1, 1)
        y_vec = np.array(y)[valid_mask]

        # ensure minimum samples per leaf
        min_samples_leaf = max(int(len(X) * min_leaf_frac), 1)

        clf = DecisionTreeClassifier(
            max_leaf_nodes=n_bins,
            min_samples_leaf=min_samples_leaf,
        )
        clf.fit(X, y_vec)
        leaf_ids = clf.apply(X)

        leaf_series = pd.Series(leaf_ids, index=s[valid_mask].index)
        bins[valid_mask] = leaf_series.map(lambda v: f"leaf_{int(v)}")
        bins[missing_mask] = "MISSING"
        return bins

    else:
        raise ValueError(f"Unsupported binning method: {method}")

tools = [bin_single_feature_tool]

agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "将特征 series 分箱，方法为 'quantile'，分箱数为 10"
        }
    ]
}

result = agent.invoke(Message, context = UserContext(user_id="user123"))
result['messages']


PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.IsInstanceSchema (<class 'pandas.core.series.Series'>)

For further information visit https://errors.pydantic.dev/2.12/u/invalid-for-json-schema

In [21]:
import pandas as pd
import numpy as np
from typing import List, Optional, Dict, Any
from sklearn.tree import DecisionTreeClassifier
from langchain.tools import tool

# ============== 1. Binning helper =====================
@tool
def bin_single_feature_tool(
    values: List[float],
    indices: Optional[List[Any]] = None,
    y_values: Optional[List[float]] = None,
    y_indices: Optional[List[Any]] = None,
    method: str = "quantile",   # 'quantile' | 'width' | 'tree'
    n_bins: int = 10,
    min_leaf_frac: float = 0.05,
) -> Dict[str, Any]:
    """
    Bin a single numeric feature according to the specified method.

    Parameters
    ----------
    values : List[float]
        List of raw feature values.
    indices : Optional[List[Any]]
        Optional list of indices corresponding to values.
    y_values : Optional[List[float]]
        Optional list of target labels (only needed when method='tree').
    y_indices : Optional[List[Any]]
        Optional list of indices corresponding to target values.
    method : str
        'quantile' (equal frequency), 'width' (equal width),
        or 'tree' (decision-tree-based optimal binning).
    n_bins : int
        Number of bins or max leaf nodes (for tree).
    min_leaf_frac : float
        Minimum fraction of samples per leaf for tree binning.

    Returns
    -------
    Dict[str, Any]
        Dictionary containing 'values' (binned values) and optional 'indices'.
        Missing values are grouped into 'MISSING'.
    """
    # 创建索引
    if indices is None:
        indices = list(range(len(values)))
    
    # 转换为pandas Series进行处理
    s = pd.Series(values, index=indices)
    s = pd.to_numeric(s, errors="coerce")
    bins = pd.Series(index=s.index, dtype="object")

    # handle missing values as a separate bin
    missing_mask = s.isna()

    # ================== quantile / width ==================
    if method in ("quantile", "width"):
        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            # all values are missing
            bins[missing_mask] = "MISSING"
            return {"values": bins.tolist(), "indices": indices}

        s_valid = s[valid_mask]

        try:
            if method == "quantile":
                # equal frequency binning
                binned = pd.qcut(s_valid, q=n_bins, duplicates="drop")
            else:
                # equal width binning
                binned = pd.cut(s_valid, bins=n_bins, duplicates="drop")
        except ValueError:
            # e.g. all values are identical
            binned = pd.Series(["ALL"] * valid_mask.sum(), index=s_valid.index)

        bins[valid_mask] = binned.astype(str)
        bins[missing_mask] = "MISSING"
        return {"values": bins.tolist(), "indices": indices}

    # ================== tree-based binning ==================
    elif method == "tree":
        if y_values is None:
            raise ValueError("y_values must be provided when method='tree'.")

        # 创建目标变量的Series
        if y_indices is None:
            y_indices = indices
        y_series = pd.Series(y_values, index=y_indices)
        
        valid_mask = ~missing_mask
        if valid_mask.sum() == 0:
            bins[missing_mask] = "MISSING"
            return {"values": bins.tolist(), "indices": indices}

        X = s[valid_mask].values.reshape(-1, 1)
        # 确保只使用与X对应的y值
        y_vec = y_series.loc[s[valid_mask].index].values

        # ensure minimum samples per leaf
        min_samples_leaf = max(int(len(X) * min_leaf_frac), 1)

        clf = DecisionTreeClassifier(
            max_leaf_nodes=n_bins,
            min_samples_leaf=min_samples_leaf,
        )
        clf.fit(X, y_vec)
        leaf_ids = clf.apply(X)

        leaf_series = pd.Series(leaf_ids, index=s[valid_mask].index)
        bins[valid_mask] = leaf_series.map(lambda v: f"leaf_{int(v)}")
        bins[missing_mask] = "MISSING"
        return {"values": bins.tolist(), "indices": indices}

    else:
        raise ValueError(f"Unsupported binning method: {method}")


tools = [bin_single_feature_tool]

agent = create_agent(
    model=model,
    tools=tools,
    context_schema=UserContext,
    system_prompt="You are a helpful assistant."
)

Message = {
    "messages": [
        {
            "role": "user", 
            "content": "将特征 series 分箱，方法为 'quantile'，分箱数为 10"
        }
    ]
}

result = agent.invoke(Message, context = UserContext(user_id="user123"))
result['messages']

[HumanMessage(content="将特征 series 分箱，方法为 'quantile'，分箱数为 10", additional_kwargs={}, response_metadata={}, id='346b25eb-e15c-42df-872e-71a9b4463fbf'),
 AIMessage(content="我需要将特征 series 进行分箱处理，使用 'quantile' 方法分成 10 个箱。不过，我需要您提供具体的数值数据才能进行分箱操作。\n\n请您提供以下信息：\n- 特征 series 的具体数值列表（values）\n\n例如，您可以提供类似这样的数据：\n```\n[1.2, 3.4, 5.6, 7.8, 9.0, 2.1, 4.3, 6.5, 8.7, 10.9, ...]\n```\n\n一旦您提供了数值数据，我就可以使用等频分箱方法将其分成10个箱。", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 525, 'total_tokens': 661, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 525}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '1a454715-4096-4e97-944f-50ecfd6c3327', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--16f2dcbe-35aa-45b0-9a01-47a65c234803-0', usage_metadata={'